In [45]:
import glob
import pathlib
import re
import pandas as pd
import numpy as np
import h5py
import cv2

In [19]:
bounding_box_df = pd.read_csv("datasets/wacv2024_ictrap_dataset/3_m-h-h_annotation.csv")

In [21]:
bounding_box_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15052 entries, 0 to 15051
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   frame_index  15052 non-null  int64  
 1   is_keyframe  15052 non-null  bool   
 2   class        15052 non-null  object 
 3   confidence   15052 non-null  object 
 4   left         15052 non-null  float64
 5   top          15052 non-null  float64
 6   width        15052 non-null  float64
 7   height       15052 non-null  float64
 8   center_x     15052 non-null  float64
 9   center_y     15052 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(2)
memory usage: 1.0+ MB


In [22]:
point_cloud_dir = "datasets/insect_combined/full_trajectories/mu-3/"
point_cloud_dir = pathlib.Path(point_cloud_dir).as_posix()

In [23]:
point_clouds = glob.glob(f"{point_cloud_dir}/*.csv")

In [24]:
# Add the end time to the filename if not already present.
# This makes it easier to skip point clouds that lay outside a frame time.

regex = re.compile(r"\d+")

for point_cloud in point_clouds:
    if not "end" in point_cloud:
        df = pd.read_csv(point_cloud)
        start = int(regex.findall(point_cloud)[-1])
        end = start + df.loc[:, 't'].max()

        file = pathlib.Path(point_cloud)
        file.rename(file.with_name(file.stem + f"_end{end}.csv"))

point_clouds = glob.glob(f"{point_cloud_dir}/*.csv")

In [25]:
# create list with unique frame indices.
frames_with_bbs = bounding_box_df.loc[:, 'frame_index'].unique()
regex = re.compile(r"\d+")
consolidated_dict = []

# Open HDF5 file containing the external triggers and the corresponding frames.
with h5py.File("./datasets/wacv2024_ictrap_dataset/3_m-h-h.h5", "r") as f:
    ext_triggers_negative = f['EXTERNAL_TRIGGERS']['corrected_negative']
    ext_triggers_positive = f['EXTERNAL_TRIGGERS']['corrected_positive']

    for frame_number in frames_with_bbs:
        start_exposure = ext_triggers_negative[ext_triggers_negative['f'] == frame_number]['t']
        end_exposure = ext_triggers_positive[ext_triggers_positive['f'] == frame_number]['t']

        df = bounding_box_df.loc[bounding_box_df['frame_index'] == frame_number]
        df_dict = df.to_dict(orient='records')

        if start_exposure.size == 0 or end_exposure.size == 0:
            df_dict[i]['class'] = "no class"
            consolidated_dict.append(df_dict[i])
            continue

        # Load the start and end exposure time for the current frame.
        start_exposure = start_exposure.item()
        end_exposure = end_exposure.item()
        # create a separate dataframe for the current frame.
        df = bounding_box_df.loc[bounding_box_df['frame_index'] == frame_number]
        df_dict = df.to_dict(orient='records')

        # Dictionary to store bounding boxe hits in
        bb_match_dict = {}

        x0 = df['left'].to_numpy()
        y0 = df['top'].to_numpy()
        x1 = x0 + df['width'].to_numpy()
        y1 = y0 + df['height'].to_numpy()

        for point_cloud in point_clouds:
            start = int(regex.findall(point_cloud)[-2])
            end = int(regex.findall(point_cloud)[-1])

            if not (start_exposure > end or end_exposure < start):
                points = pd.read_csv(point_cloud)

                # Only take points that fall inside the exposure time of the selected frame
                points = points[((points['t'] + start) >= start_exposure) & ((points['t'] + start) <= end_exposure)]

                # Extract the x and y coordinate columns
                points = points.loc[:, ['x', 'y']].to_numpy()
                if points.size == 0:
                    continue
                # Create an array for the x and y coordinates in shape of (n,1)
                points_x = points[:, 0][:, None]
                points_y = points[:, 1][:, None]

                matches = pd.DataFrame((points_x >= x0) & (points_x <= x1) & (points_y >= y0) & (points_y <= y1))
                matches = matches.apply(pd.Series.value_counts)
                bb_match_dict[point_cloud] = matches.replace(np.nan, 0)

        for i in range(len(df.index)):
            counter = ["", 0]
            for key, value in bb_match_dict.items():
                if True in value[i] and int(value[i][True]) > counter[1]:
                    counter[0] = key
                    counter[1] = value[i][True]

            # Correct the class to the correct insect
            if not counter[0]:
                df_dict[i]['class'] = "no class"
            else:
                df_dict[i]['class'] = counter[0].split("_")[-4]
            consolidated_dict.append(df_dict[i])



In [26]:
consolidated_df = pd.DataFrame(consolidated_dict)
consolidated_df.to_csv("./datasets/consolidated_data.csv", index=False)

In [35]:
consolidated_df.loc[:, 'class'].value_counts()

class
bee         11379
no class     2862
ins           810
Name: count, dtype: int64

In [41]:
consolidated_df[consolidated_df['is_keyframe'] == True].loc[:, 'class'].value_counts()

class
bee         3172
ins          530
no class      33
Name: count, dtype: int64

In [ ]:
vidcap = cv2.VideoCapture("./datasets/wacv2024_ictrap_dataset/3_m-h-h_dvs.mp4")
success,image = vidcap.read()
count = 0

while success:
    cv2.imwrite(f"./datasets/insects/frame{count}.jpg", image)
    success,image = vidcap.read()
    count += 1
